In [59]:
import requests

#api_key='gaKMkjL1JzRhLGQ8pofFgbAN8wRMQNHcT0G96xr5'
from google.colab import userdata


search_query = 'cottage cheese 2%'
api_key = userdata.get('USDA_KEY')

# Step 1: Search for foods by keyword (e.g., "chicken")
search_url = "https://api.nal.usda.gov/fdc/v1/foods/search"
params = {
    'query': search_query,  # Search term
    'api_key': api_key
}

# Perform the search
response = requests.get(search_url, params=params)
data = response.json()



In [60]:
def QueryUSDA(query_block, api_key):
  query_list = query_block.split('\n')
  data_coll={}
  for query in query_list:
    print(query)
    search_url = "https://api.nal.usda.gov/fdc/v1/foods/search"
    params = {
        'query': query,  # Search term
        'api_key': api_key
    }

    # Perform the search
    response = requests.get(search_url, params=params)
    data = response.json()
    data_coll[query] = data
  return data_coll

In [61]:
USDA_data=QueryUSDA('cottage cheese 2% \n chicken', api_key)


cottage cheese 2% 
 chicken


In [62]:
for item in USDA_data:
  print(item)

cottage cheese 2% 
 chicken


In [63]:
import pandas as pd
import numpy as np


In [64]:
# Extract the first food item
food_item = data['foods'][0]

# Extract the description
description = food_item['description']

# Extract the list of nutrients
food_nutrients = food_item['foodNutrients']
# Create a dictionary with nutrient names as keys and their values
nutrient_data = {nutrient['nutrientName']: nutrient['value'] for nutrient in food_nutrients}


df = pd.DataFrame(nutrient_data, index=[description])
df
# List of target nutrients
target_nutrients = ['Protein', 'Carbohydrate, by difference', 'Total lipid (fat)', 'Fiber, total dietary']
# Add missing nutrients with NaN values
for nutrient in target_nutrients:
    if nutrient not in df.columns:
        df[nutrient] = 0
columns = ['Ingredient']+['Calories']+target_nutrients
combined_df = pd.DataFrame(columns=columns)



In [68]:
# Define the target nutrients and columns for the final DataFrame
target_nutrients = ['Protein', 'Carbohydrate, by difference', 'Total lipid (fat)', 'Fiber, total dietary']
columns = ['Ingredient', 'Calories'] + target_nutrients

# Initialize empty DataFrame
combined_df = pd.DataFrame(columns=columns).set_index('Ingredient')

# Loop through USDA data
for item in USDA_data:
    food_item = USDA_data[item]['foods'][0]

    # Extract description and nutrients
    description = food_item['description']
    food_nutrients = food_item['foodNutrients']

    # Build nutrient dict
    nutrient_data = {n['nutrientName']: n['value'] for n in food_nutrients}

    # Ensure all target nutrients are present
    for nutrient in target_nutrients:
        if nutrient not in nutrient_data:
            nutrient_data[nutrient] = 0

    # Compute calories
    calories = (
        nutrient_data['Protein'] * 4 +
        nutrient_data['Carbohydrate, by difference'] * 4 +
        nutrient_data['Total lipid (fat)'] * 9 +
        nutrient_data['Fiber, total dietary'] * 2
    )

    # Construct row data
    row_data = {
        'Calories': calories,
        'Protein': nutrient_data['Protein'],
        'Carbohydrate, by difference': nutrient_data['Carbohydrate, by difference'],
        'Total lipid (fat)': nutrient_data['Total lipid (fat)'],
        'Fiber, total dietary': nutrient_data['Fiber, total dietary']
    }

    # Append row to DataFrame
    combined_df.loc[description] = row_data

# Show result
combined_df


,Calories,Protein,"Carbohydrate, by difference",Total lipid (fat),"Fiber, total dietary"
Ingredient,,,,,
"Cheese, cottage, lowfat, 2% milkfat",81.94,11.0,4.31,2.30,0.0
CHICKEN,101.71,21.4,0.00,1.79,0.0


In [70]:
print(combined_df['Protein'].sum())

32.4


------------------------------------- individual query by fdc id ------------------------

In [ ]:
# Step 2: Assume the user selects a row (e.g., row 1) from the list
# Select the first food item (or any other based on user input)
selected_food = data['foods'][0]  # In this case, selecting the first item
fdc_id = selected_food['fdcId']  # Get the FDC ID of the selected food

print(f"\nFetching nutrition details for: {selected_food['description']} (FDC ID: {fdc_id})")

# Step 3: Fetch detailed nutrition info using the FDC ID
details_url = f"https://api.nal.usda.gov/fdc/v1/food/{fdc_id}"
details_response = requests.get(details_url, params={'api_key': api_key})
food_details = details_response.json()

# Step 4: Display detailed nutrition info

print(f"Name: {food_details['description']}")



In [ ]:
# Step 2: Assume the user selects a row (e.g., row 1) from the list
# Select the first food item (or any other based on user input)
selected_food = data['foods'][0]  # In this case, selecting the first item
fdc_id = selected_food['fdcId']  # Get the FDC ID of the selected food

print(f"\nFetching nutrition details for: {selected_food['description']} (FDC ID: {fdc_id})")

# Step 3: Fetch detailed nutrition info using the FDC ID
details_url = f"https://api.nal.usda.gov/fdc/v1/food/{fdc_id}"
details_response = requests.get(details_url, params={'api_key': api_key})
food_details = details_response.json()

# Step 4: Display detailed nutrition info

print(f"Name: {food_details['description']}")



In [ ]:
food_nutrients

In [ ]:
food_details

In [ ]:
unit_dict = {
    "g": 1,
    "mg": 0.001,
    "µg": 0.000001,
    "kcal": 0.00000001,
    "kJ": 0.00000001,
    "IU": 0.0000001
}

tot_amt=0
for nutrient in food_details['foodNutrients']:
    #print(nutrient)
    #print(nutrient['nutrient']['name'])

    if 'amount' in nutrient:
        nutrient_amt=nutrient['amount']
        nutrient_unit=nutrient['nutrient']['unitName']
        nutrient_name=nutrient['nutrient']['name']
        print(f"{nutrient_name}: {nutrient_amt} {nutrient_unit}")
        tot_amt += float(nutrient['amount']) * unit_dict[nutrient_unit]
        print(f"sub-total: {tot_amt}")

print(f"Total: {tot_amt}")
